In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Graph contractions


- Can we make graph search more parallelizable?
  - E.g., $\lg n$ span?
  
- We'd like to do divide and conquer, but this won't work. Why?

- Instead, we'll do contraction (recall contraction implementation for scan)

- Let's take the problem of Graph Connectivity:
- Graph Connectivity
  - recall we solved this in lab 10
  (figure)
  - 
  - BFS has span proportional to diameter
      - need to solve each connected component sequentially

- Graph partitioning
  - subgraphs
  - vertex-induced subgraphs
  - 

- The graph connectivity problem is to partition an undirected graph into its components (maximal connected subgraphs)


- Graph contraction
  - shrink size of graph and solve connectivity of that smaller graph
  - solve different components in parallel
  - similar idea to divide and conquer, but for graphs
  - we'll also use this idea to solve other graph problems, like spanning trees
  
- contract: $V \mapsto V^1, V^2$
  - $V_1, V_2$ are partitions of the graph
  - each is connected, but may not be maximal (figure)
  - use a "representative node" for each partition 
  - contract code
  
Types of contraction
- Edge Contraction: Only pairs of vertices connected by an edge are contracted.
- Star Contraction: Vertices around a “center star” collapse to the “star”
- Tree Contraction: disjoint trees within the graph are identified and vertices in a tree are collapsed to the root.

Edge contraction
- find "disjoint" edges -- no vertices shared.


- walk through example

- show that it fails on stars. for tomorrow.

Lab: do a edge contraction and/or graph partitioning